![](check_point_logo.png)

# Tutorial on manipulating products with EOPF dev
* [OLCI L1 EFR](#olci-l1-efr)
    * [Open product](#open-the-product-using-eopf-api-and-visualize-the-tree-structure)
    * [Open measurement data](#opening-measurement-data)
    * [Plotting](#simple-raster-plot)
    * [Interpolating data](#open-meteorological-conditions)
* [SLSTR FRP](#slstr-frp)
    * [Open product](#slstr-frp)
    * [Plot active fire pixels](#plot-active-fire-pixels-positions-on-a-platecarree-grid)
* [SYN AOD](#syn-aod)
    * [Open product](#syn-aod)
    * [Plot geolocated data](#plot-using-the-coordinates-lonlat)



In [ ]:
from pathlib import Path
from eopf.store.safe2 import XarrayStore
from eopf.store.safe2 import EOSafeStore2
from eopf.common.constants import OpeningMode

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

Change directory to EOPF CPM source code ....

In [ ]:
import os
path = '/mount/internal/work-st/projects/cs-412/2078-dpr/workspace/vlevasseur/Projects/eopf-cpm'
old_path = os.getcwd()
os.chdir(path)


# OLCI L1 EFR
Product can only be accessed locally

In [ ]:
SAMPLE_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
files = [ f for f in SAMPLE_PATH.glob("S03OLCEFR*.zarr")]
product = files[0]
product

### Open the product using EOPF API and visualize the tree structure

In [ ]:
xs = XarrayStore(str(product))
xs.open()
eop=xs.load()
eop


### Opening measurement data

In [ ]:
rad = eop.measurements.image.oa01_radiance
rad._data.encoding

Underlying data is dask.array

In [ ]:
rad.data

In [ ]:
rad.attrs

In [ ]:
rad.is_scaled
rad.is_masked
rad.sel()

#### Simple raster plot
Note that using xarray, data is correctly decoded and masked

In [ ]:
rad.plot()

#### Plot using the coordinates (lon,lat)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 6))
ax = plt.axes()
rad._data.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

#### Open Meteorological conditions

In [ ]:
meteo=eop.conditions.meteo

#### Interpolate the atmo. temp. profile at p=832.2 hPa and plot

In [ ]:
tp=meteo["atmospheric_temperature_profile"]._data.interp(tie_pressure_levels=832.2)
tp.plot()

#### Open sat/sun angles
Angles are stored on a tiepoint subgrid

In [ ]:
ds = eop.conditions.geometry
ds

#### Have a look at condition parameters for removed pixels

In [ ]:
ds = eop.conditions.orphans
ds

# SLSTR FRP

In [ ]:
SAMPLE_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
files = [ f for f in SAMPLE_PATH.glob("S03SLSFRP*.zarr")]
product = files[1]
product

In [ ]:
xs = XarrayStore(str(product))
xs.open()
eop=xs.load()
eop

### Opening measurement data (1D)
Cannot find the lat/lon in this product ....

In [ ]:
meas_in = eop.measurements.inadir
meas_in

### Plot Active Fire pixels positions on a PlateCarree grid

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
# ax.set_global()
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.scatter(meas_in.data.longitude,meas_in.data.latitude,c=meas_in.frp_mwir,vmax=100)
plt.colorbar()


# SYN AOD

In [ ]:
SAMPLE_PATH = Path("/mount/internal/work-st/projects/cs-412/2078-dpr/Samples/Products/Zarr_Beta")
files = [ f for f in SAMPLE_PATH.glob("S03SYNAOD*.zarr")]
product = files[0]
product

In [ ]:
xs = XarrayStore(str(product))
xs.open()
eop=xs.load()
eop

In [ ]:
aod550 = eop.measurements.aod_550
aod550


In [ ]:
aod550.plot()

#### Plot using the coordinates (lon,lat)
Note that in SYN AOD product, lat/lon are undefined when the data is missing, which is not correctly handled by matplotlib pcolormesh

In [ ]:
# Remove margins
aod550_dropna=aod550._data.dropna("columns",how="all")
aod550_dropna=aod550_dropna.dropna("rows",how="all")

# Fill remaining missing values
aod550_dropna["latitude"] = (["rows","columns"],aod550_dropna.latitude.bfill("columns").data)
aod550_dropna["longitude"] = (["rows","columns"],aod550_dropna.longitude.bfill("columns").data)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 6))
ax = plt.axes()
aod550_dropna.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)